In [61]:
# imports

import os
import requests
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display

In [43]:
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')


if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set (and this is optional)")

OpenAI API Key exists and begins sk-proj-
Google API Key exists and begins AI


In [44]:
# Connect to OpenAI client library
# A thin wrapper around calls to HTTP endpoints

openai = OpenAI()

# For Gemini, DeepSeek and Groq, we can use the OpenAI python client
# Because Google and DeepSeek have endpoints compatible with OpenAI
# And OpenAI allows you to change the base_url

gemini_url = "https://generativelanguage.googleapis.com/v1beta/openai/"
ollama_url = "http://localhost:11434/v1"

gemini = OpenAI(api_key=google_api_key, base_url=gemini_url)
ollama = OpenAI(api_key="ollama", base_url=ollama_url)

In [49]:
tell_a_joke = [
    {"role": "user", "content": "Tell a joke for a student on the journey to becoming an expert in LLM Engineering"},
]

In [50]:
response = openai.chat.completions.create(model="gpt-4.1-mini", messages=tell_a_joke)
display(Markdown(response.choices[0].message.content))

Why did the LLM engineering student bring a ladder to the AI lab?

Because they heard the models were on another level!

In [32]:
models = gemini.models.list()
for model in models:
    print(model.id)

models/gemini-2.5-flash
models/gemini-2.5-pro
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-exp-1206
models/gemini-2.5-flash-preview-tts
models/gemini-2.5-pro-preview-tts
models/gemma-3-1b-it
models/gemma-3-4b-it
models/gemma-3-12b-it
models/gemma-3-27b-it
models/gemma-3n-e4b-it
models/gemma-3n-e2b-it
models/gemini-flash-latest
models/gemini-flash-lite-latest
models/gemini-pro-latest
models/gemini-2.5-flash-lite
models/gemini-2.5-flash-image
models/gemini-2.5-flash-preview-09-2025
models/gemini-2.5-flash-lite-preview-09-2025
models/gemini-3-pro-preview
models/gemini-3-flash-preview
models/gemini-3-pro-image-preview
models/nano-banana-pro-preview
models/gemini-robotics-er-1.5-preview
models/gemini-2.5-computer-use-preview-10-2025
models/deep-research-pro-preview-12-2025
models/gemini-embedding-001
models/aqa
models/imagen-4.0-generate-preview-06-06
models/imagen

In [ ]:
response = gemini.chat.completions.create(model="ggemma-3-1b-it", messages=tell_a_joke)
display(Markdown(response.choices[0].message.content))

In [51]:
easy_puzzle = [
    {"role": "user", "content": 
        "You toss 2 coins. One of them is heads. What's the probability the other is tails? Answer with the probability only."},
]

In [52]:
response = openai.chat.completions.create(model="gpt-5-nano", messages=easy_puzzle, reasoning_effort="minimal")
display(Markdown(response.choices[0].message.content))

1/3

In [53]:
response = openai.chat.completions.create(model="gpt-5-nano", messages=easy_puzzle, reasoning_effort="low")
display(Markdown(response.choices[0].message.content))

2/3

In [54]:
response = openai.chat.completions.create(model="gpt-5-mini", messages=easy_puzzle, reasoning_effort="minimal")
display(Markdown(response.choices[0].message.content))

NotFoundError: Error code: 404 - {'error': {'message': 'Your organization must be verified to use the model `gpt-5-mini`. Please go to: https://platform.openai.com/settings/organization/general and click on Verify Organization. If you just verified, it can take up to 15 minutes for access to propagate.', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_found'}}

In [56]:
hard = """
On a bookshelf, two volumes of Pushkin stand side by side: the first and the second.
The pages of each volume together have a thickness of 2 cm, and each cover is 2 mm thick.
A worm gnawed (perpendicular to the pages) from the first page of the first volume to the last page of the second volume.
What distance did it gnaw through?
"""
hard_puzzle = [
    {"role": "user", "content": hard}
]

In [57]:
response = openai.chat.completions.create(model="gpt-5-nano", messages=hard_puzzle, reasoning_effort="minimal")
display(Markdown(response.choices[0].message.content))


Interpretation: You have two adjacent books on a shelf, with their pages and covers described. Each volume has:
- pages thickness: 2 cm total per volume
- each cover thickness: 2 mm (0.2 cm)
- normally, a book has: front cover, pages, back cover in that order.

When the two volumes stand side by side, the order (left to right) is:
Volume 1 (V1): [ front cover1, pages1, back cover1 ] then Volume 2 (V2): [ front cover2, pages2, back cover2 ].

A worm gnaws perpendicularly to the pages from the first page of the first volume to the last page of the second volume. That means the worm starts at the very first page inside V1 (i.e., just after the front cover of V1) and ends at the very last page inside V2 (i.e., just before the back cover of V2). The worm’s path passes through: the remaining interior between those pages, which includes the back cover of V1, the gap between the books (the touching faces), and the front cover of V2, plus possibly the inner pages near the ends.

Key observation: The thicknesses of pages given are per volume, and each cover is 0.2 cm. The worm starts at the first page of V1 (so just after the front cover of V1) and ends at the last page of V2 (just before the back cover of V2). The distance the worm travels is the total thickness from that starting plane to that ending plane through the intervening materials.

Compute segment by segment from left to right:
- Inside V1: it starts at the first page, which is immediately after the front cover. It must gnaw through the rest of V1’s pages and the back cover to reach the outside of V1 toward V2. However, because it starts at the first page, we should not count any of the pages before it (there are none). It must go through the remainder of V1 pages and V1 back cover.
Total thickness of V1 to traverse from first page to the right outside face toward V2:
- remaining pages in V1: since the total pages thickness is 2 cm, and he starts at the first page, the distance through V1 from first page to the back of V1 is essentially the thickness of the rest of the pages plus the back cover. The first page thickness contribution is negligible; the model usually simplifies to: gnaw through the rest of V1 pages (almost all 2 cm) plus V1 back cover (0.2 cm).
But since the first page is right after the front cover, the distance from that first page to the back cover of V1 is: pages remaining ≈ 2 cm (since the first page is just a boundary) plus back cover 0.2 cm. In standard puzzle treatment, the worm would have to go through the rest of V1’s pages (approximately 2 cm) plus V1 back cover (0.2 cm).

- Gap between volumes: the touching faces of the two volumes: there is no gap; the books are flush. The worm must pass through the front cover of V2 to reach the pages of V2? Actually to reach the last page of V2, the worm goes through the front cover of V2 (0.2 cm) and into the pages, ending at the last page. Since it ends at the last page, it does not go through the back cover of V2.

Thus, total distance = (V1 remaining pages) + (V1 back cover) + (V2 front cover) + (V2 pages up to last page).

But the insect ends at the last page of V2, which is just before the back cover; thus it must go through all of V2 pages.

So sum:
- V1 remaining pages: approximately 2 cm
- V1 back cover: 0.2 cm
- V2 front cover: 0.2 cm
- V2 pages: 2 cm

Total = 2 cm + 0.2 cm + 0.2 cm + 2 cm = 4.4 cm

Answer: 4.4 cm.

In [58]:
response = openai.chat.completions.create(model="gpt-5", messages=hard_puzzle)
display(Markdown(response.choices[0].message.content))

NotFoundError: Error code: 404 - {'error': {'message': 'Your organization must be verified to use the model `gpt-5`. Please go to: https://platform.openai.com/settings/organization/general and click on Verify Organization. If you just verified, it can take up to 15 minutes for access to propagate.', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_found'}}

In [60]:
response = gemini.chat.completions.create(model="gemini-2.0-flash", messages=hard_puzzle)
display(Markdown(response.choices[0].message.content))

RateLimitError: Error code: 429 - [{'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/rate-limit. \n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_input_token_count, limit: 0, model: gemini-2.0-flash\n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 0, model: gemini-2.0-flash\n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 0, model: gemini-2.0-flash\nPlease retry in 20.661939612s.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.goog